# Исследование Postgres

In [1]:
import time
import statistics

class TimerCode:
    def __init__(self):
        self.start = time.time()
        self.durations = []

    def setup_start_time(self):
        self.start = time.time()

    def checkpoint(self):
        end_time = time.time()
        self.durations.append(end_time - self.start)
        self.start = end_time

    def get_current_stat(self):
        print(f"Median - {statistics.median(self.durations)}")
        avg = statistics.mean(self.durations)
        print(f"Average - {avg}")
        print(f"Summary - {sum(self.durations)}")

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.get_current_stat()

In [ ]:
import psycopg2
import uuid
import random

# Define the database connection parameters
db_params = {
    'host': 'localhost',
    'port': 5432,
    'dbname': 'db',
    'user': 'user',
    'password': 'password'
}

# Create the connection and cursor objects
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Create the users table
cur.execute("""
    CREATE TABLE users (
        id UUID PRIMARY KEY,
        name VARCHAR(255) NOT NULL
    );
""")

# Create the movies table
cur.execute("""
    CREATE TABLE movies (
        id UUID PRIMARY KEY,
        title VARCHAR(255) NOT NULL
    );
""")

# Create the bookmarks table
cur.execute("""
    CREATE TABLE bookmarks (
        user_id UUID REFERENCES users(id),
        movie_id UUID REFERENCES movies(id),
        PRIMARY KEY (user_id, movie_id)
    );
""")

# Generate some fake user and movie ids
USERS_NUM = 500_000
MOVIES_NUM = 30_000

BOOKMARKS_PER_USER = 40

user_ids = [str(uuid.uuid4()) for _ in range(USERS_NUM)]
movie_ids = [str(uuid.uuid4()) for _ in range(MOVIES_NUM)]

# Insert the users and movies into the database
for user_id in user_ids:
    cur.execute("INSERT INTO users (id, name) VALUES (%s, 'User %s')", (user_id, user_id))

for movie_id in movie_ids:
    cur.execute("INSERT INTO movies (id, title) VALUES (%s, 'Movie %s')", (movie_id, movie_id))

# Create the bookmarks for each user
for user_id in user_ids:
    bookmarks = [
        movie_id for movie_id in random.sample(movie_ids, BOOKMARKS_PER_USER)
    ]
    for movie_id in bookmarks:
        cur.execute("INSERT INTO bookmarks (user_id, movie_id) VALUES (%s, %s)", (user_id, movie_id))

# Commit the changes and close the cursor and connection
conn.commit()
cur.close()
conn.close()